Goal:

1. Create new AWS instance with Mongo and anaconda. 

2. Create function which take urlname, date (with default set to Jan 1, 2018, and inserts these things to Mongo:

    
Let's make two databases, one on group, another on events. 

Or the function could take an area and a date. 

## Austin:

I say central zipcode is 78702. 12 miles seems to encompass the city nicely. Let's do 17 to be safe. Well, when you think about it, don't want to get meetups too far away.



Questions to answer:
Are closed groups, do they have higher retentions? Or approval?

There is the events\:id\attendance api call. However, are people taking attendance? Out of the three groups I looked at
originally, 1 was. And it seems they mark when people are no-shows.

So I can do this call:
https://api.meetup.com/Hiking-for-Singles/events/251917488/attendance?&sign=true&photo-host=public&page=20&only=member.id,attendance_id,status,rsvp.response

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import requests
import json
import time
import sys

sys.path.append('./src')
key = open('src/api.txt', 'r').read()

client = MongoClient()
db = client.meetup

In [10]:
def get_attendees_for_group_and_time(zipcode, radius):
    '''
    Finds all groups within a certain region. If their visibility is set to public, 
    insert them into the mongo groups collection in the meetup db. 
    
    Then the function gets the events for each group and gets the attendees for the event, inserts 
    that into event_attendees db. 
    
    Parameters:
    zipcode (int): Center of search for groups.
    radius(int): Distance past zip to search for groups.

    Returns (in groups db):

    1. GroupID
    2. Status
    3. urlname
    4. Date created
    5. City
    6. State
    7. join_mode- This can be approval, closed, or open. 
    8. Visibility 
    9. Lat 
    10. Lon 
    11. # of members
    12. who (what they call members)
    13. Key_photo
    14. category.id 
    15. topics
    16. contributions: can people 'contribute' money? 
    17. Plain text description 
    18. join_info.questions, join_info.photo_req, join_info.questions_req
    19. past_event_count
    20. meta_category.id 
    21. membership_dues.required, membership_dues.fee, membership_dues.fee_desc (How often fee due)
    
    Returns (in events db):
    
    
    '''
    req = requests.get('https://api.meetup.com/find/groups?key={}&zip={}&radius={}\
    &fields=join_info,contributions,discussion_preferences,membership_dues,\
    best_topics,past_event_count,topics,plain_text_description&page=200&offset={}\
    &omit=link,score,name,description,country,organizer,timezone,next_event,\
    ,localized_country_name,localized_location,untranslated_city,meta_category.photo,\
    ,key_photo.thumb_link,key_photo.base_url,topics.lang'
                       .format(key, zipcode, radius, 0))
    json_data = json.loads(req.text)
    db.groups.insert_many(json_data)
    #This will give us the number of groups.
    number_of_groups = int(req.headers['X-Total-Count'])
    offsets = list(range(int(number_of_groups/200) + 1))
    
    return number_of_groups, offsets
    
get_attendees_for_group_and_time(78702, 12)

(2719, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])

In [ ]:
def get_events_for_group(starting_date):
    pass